# this version is to model historical - W
# data: 20221102


In [46]:
import numpy as np
import pandas as pd
import configparser

conf = '../configuration/PEMSD7M.conf'
config = configparser.ConfigParser()
config.read(conf)


['../configuration/PEMSD7M.conf']

In [47]:
config_data = config['Data']
FLOWPATH = config_data['FLOWPATH']
DATANAME = config_data['DATANAME']
N_NODE = int(config_data['N_NODE'])
DATANAME = config_data['DATANAME']

In [48]:
if DATANAME == 'PEMS04':
    data = np.squeeze(np.load(FLOWPATH)['data'])[...,0]
    index = pd.date_range(start = '2018-01-01',end='2018-03-01', freq='5min')[:-1]
    data = pd.DataFrame(data)
    data.index = index
elif DATANAME == 'PEMS03':
    data = np.squeeze(np.load(FLOWPATH)['data'])
    index = pd.date_range(start = '2018-09-01',end='2018-12-01', freq='5min')[:-1]
    data = pd.DataFrame(data)
    data.index = index
elif DATANAME == 'PEMS07':
    data = np.squeeze(np.load(FLOWPATH)['data'])
    index = pd.date_range(start = '2017-05-01',end='2017-08-07', freq='5min')[:-1]
    data = pd.DataFrame(data)
    data.index = index
elif DATANAME == 'PEMS08':
    data = np.squeeze(np.load(FLOWPATH)['data'])[...,0]
    index = pd.date_range(start = '2016-07-01',end='2016-09-01', freq='5min')[:-1]
    data = pd.DataFrame(data)
    data.index = index
elif DATANAME == 'METR-LA':
    data = pd.read_hdf(FLOWPATH)
    data = pd.DataFrame(data)
elif DATANAME == 'PEMS-BAY':
    data = pd.read_hdf(FLOWPATH)
    data = pd.DataFrame(data)
elif DATANAME == 'PEMSD7M':
    data = pd.read_csv(FLOWPATH,index_col=[0])
    data = pd.DataFrame(data)

In [49]:
# train data最好取整数周
sum_week = int(len(data)/288/7)

In [50]:
# split data
len_sample = 288* int(sum_week*0.7)*7
# in case there is less than 288
len_sample_v = 288* int(sum_week*0.3)*7
train_data = data[:len_sample]
test_data = data[-len_sample_v:]
# index time
time = pd.DatetimeIndex(train_data.index).weekday

In [51]:
i_week = time.unique()
sort_data = []
mean = []
std = []
for i in i_week:
    temp_data_ = np.array(train_data.iloc[time.isin([i]),:]).reshape(-1,288, N_NODE)
    temp_data_mean = np.mean(temp_data_,axis=0)
    temp_data_std = np.std(temp_data_,axis=0)
    mean.append(temp_data_mean)
    std.append(temp_data_std)
train_mean = np.stack(mean)
train_std = np.stack(std)
# sort_data = np.squeeze(np.stack(sort_data))

In [52]:
# 求得train data的上下边界
upper = train_mean + train_std
lower = train_mean - train_std
# 构造predict train的上下边界
test_time = pd.DatetimeIndex(test_data.index).weekday
test_week_index = np.array(test_time).reshape(-1,288)[:,0]
y_upper = np.stack([upper[test_week_index[i]] for i in test_week_index])
y_lower = np.stack([lower[test_week_index[i]] for i in test_week_index])

In [53]:
test_data = np.array(test_data).reshape(-1,288,N_NODE)

In [54]:
# 计算coverage和interval；
def coverage(y_lower, y_upper, test_data):
    y_lower = y_lower * (y_lower >0)
    mask = test_data > 0
    independent_coverage = np.logical_and(np.logical_and(y_upper >= test_data, y_lower <= test_data), mask)
    width = np.sum(np.abs(y_upper-y_lower)*mask)/np.sum(mask)
    m_cov = np.sum(independent_coverage)/np.sum(mask)
    return  m_cov, width

In [55]:
# historical - W
m_cov_W, width_W = coverage(y_lower, y_upper, test_data)

In [56]:
# for historical-D
D_train = np.array(train_data).reshape(-1,288, N_NODE)
mean_D = np.mean(D_train,axis=0)
std_D = np.std(D_train, axis=0)
# 求得train data的上下边界
upper_D = mean_D + std_D
lower_D = mean_D - std_D
# 构造predict train的上下边界
y_upper_D = np.tile(upper_D, reps = (test_data.shape[0],1,1))
y_lower_D = np.tile(lower_D, reps = (test_data.shape[0],1,1))

In [57]:
m_cov_D, width_D = coverage(y_lower_D, y_upper_D, test_data)

In [58]:
m_cov_D, width_D

(0.747145641882484, 10.718489497181954)

In [59]:
m_cov_W, width_W

(0.5497445871623503, 7.859360701284957)